In [1]:
from datasets import Dataset

reviews = {
    "text": [
        "I love this product. It works great!",
        "Terrible experience. Would not buy again.",
        "Excellent quality and fast shipping.",
        "Not what I expected. Pretty disappointed.",
        "Fantastic! Will definitely recommend to friends.",
        "The item arrived broken. Very unhappy.",
        "Decent for the price. Not bad.",
        "Amazing service and very user-friendly.",
        "It stopped working after a week. Waste of money.",
        "Exactly as described. Satisfied with the purchase."
    ]
}

dataset = Dataset.from_dict(reviews)
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [3]:
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-small-review-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=1,
    learning_rate=5e-5,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-4-3488839312.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kk1455217 (kk1455217-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=40, training_loss=1.2239014625549316, metrics={'train_runtime': 619.6205, 'train_samples_per_second': 0.129, 'train_steps_per_second': 0.065, 'total_flos': 2612920320000.0, 'train_loss': 1.2239014625549316, 'epoch': 10.0})

In [20]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

output = generator(
    "The product was",
    max_new_tokens=30,
    do_sample=True,
    top_k=70,
    top_p=0.95,
    temperature=0.8,
    num_return_sequences=1
)[0]['generated_text']
print("Generated Review:\n", output)



Device set to use cpu


Generated Review:
 The product was discontinued.

Great product. Will return this product great value

Perfect. Will definitely recommend to friends.

Questions & Answers




In [19]:
output = generator("The product that i bought", max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    num_return_sequences=1)[0]['generated_text']
print("Generated Review:\n", output)


Generated Review:
 The product that i bought. Pretty happy. Love it. Will definitely buy again. Thanks!
